In [1]:
import numpy as np #supporting multi-dimensional arrays and matrices
import os #read or write a file

#import cv2
from skimage import color
from skimage import io
from skimage.transform import resize
import pandas as pd #data manipulation and analysis
from tqdm import tqdm # for  well-established ProgressBar
from random import shuffle #only shuffles the array along the first axis of a multi-dimensional array. The order of sub-arrays is changed but their contents remains the same.
LR = 1e-3
MODEL_NAME = 'plantclassfication-{}-{}.model'.format(LR, '2conv-basic') # just so we remember which save

In [2]:
data_dir = './data/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
IMG_SIZE = 16

In [3]:
CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
NUM_CATEGORIES = len(CATEGORIES)
print (NUM_CATEGORIES)

12


In [4]:
def label_img(word_label):                       
    if word_label == 'Black-grass': return [1,0,0,0,0,0,0,0,0,0,0,0]
    elif word_label == 'Charlock': return [0,1,0,0,0,0,0,0,0,0,0,0]
    elif word_label == 'Cleavers': return [0,0,1,0,0,0,0,0,0,0,0,0]
    elif word_label == 'Common Chickweed': return [0,0,0,1,0,0,0,0,0,0,0,0]
    elif word_label == 'Common wheat': return [0,0,0,0,1,0,0,0,0,0,0,0]
    elif word_label == 'Fat Hen': return [0,0,0,0,0,1,0,0,0,0,0,0]
    elif word_label == 'Loose Silky-bent': return [0,0,0,0,0,0,1,0,0,0,0,0]
    elif word_label == 'Maize': return [0,0,0,0,0,0,0,1,0,0,0,0]
    elif word_label == 'Scentless Mayweed': return [0,0,0,0,0,0,0,0,1,0,0,0]
    elif word_label == 'Shepherds Purse': return [0,0,0,0,0,0,0,0,0,1,0,0]
    elif word_label == 'Small-flowered Cranesbill': return [0,0,0,0,0,0,0,0,0,0,1,0]
    elif word_label == 'Sugar beet': return [0,0,0,0,0,0,0,0,0,0,0,1] 

In [5]:
def create_train_data():
    train = []
    for category_id, category in enumerate(CATEGORIES):
        for img in tqdm(os.listdir(os.path.join(train_dir, category))):
            label=label_img(category)
            path=os.path.join(train_dir,category,img)
            #img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            #img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            img = resize(color.rgb2gray(io.imread(path)),(IMG_SIZE,IMG_SIZE))
            train.append([np.array(img),np.array(label)])
    shuffle(train)
    return train


In [6]:
train_data = create_train_data()

  0%|          | 0/263 [00:00<?, ?it/s]/home/axy/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
100%|██████████| 385/385 [00:19<00:00, 19.91it/s]


In [7]:
train_data[0][0].shape

(16, 16)

In [8]:
train = train_data

In [9]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [10]:
X.shape

(4750, 16, 16, 1)

In [11]:
Y = np.array([i[1] for i in train])

In [12]:
import tensorflow as tf

/home/axy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
x = tf.placeholder(tf.float32, shape=[None,IMG_SIZE,IMG_SIZE,1])
y_ = tf.placeholder(tf.float32, shape=[None, 12])

In [14]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [15]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


In [16]:
X1=X[:50,:,:,:]
Y1=Y[:50,:]

In [17]:
W_conv1 = weight_variable([3, 3, 1, 16])
b_conv1 = bias_variable([16])

h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([3, 3, 16, 32])
b_conv2 = bias_variable([32])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_conv3 = weight_variable([3, 3, 32, 16])
b_conv3 = bias_variable([16])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

W_conv4 = weight_variable([3, 3, 16, 32])
b_conv4 = bias_variable([32])

h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4)
h_pool4 = max_pool_2x2(h_conv4)

W_fc1 = weight_variable([32, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool4, [-1, 32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 12])
b_fc2 = bias_variable([12])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


In [18]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        for j in range(90):
            if j % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={
          x: X[j*50:j*50+50,:,:,:], y_: Y[j*50:j*50+50,:], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
            train_step.run(feed_dict={x:X[j*50:j*50+50,:,:,:], y_:Y[j*50:j*50+50,:], keep_prob: 0.5})
    save_path = saver.save(sess, "./checkpoints/model.ckpt")
    print("Model saved in path: %s" % save_path)
    writer = tf.summary.FileWriter("./checkpoints", sess.graph)

#print('test accuracy %g' % accuracy.eval(feed_dict={
      #x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
step 0, training accuracy 0.14
st

step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, training accuracy 0.16
step 3, 

step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, training accuracy 0.16
step 6, 

step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, training accuracy 0.16
step 9, 

step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12, training accuracy 0.14
step 12,

...

step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35, training accuracy 0.18
step 35,

step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38, training accuracy 0.18
step 38,

step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41, training accuracy 0.2
step 41,

step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44, training accuracy 0.22
step 44,

step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47, training accuracy 0.24
step 47,

In [19]:
z=np.reshape(X[50,:,:,:],(1,16,16,1))

In [20]:
Y[50]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [21]:
sess=tf.Session() 
saver = tf.train.import_meta_graph('./checkpoints/model.ckpt.meta')

In [22]:
import matplotlib.pyplot as plt

In [23]:
with tf.Session() as sess:
    saver.restore(sess, "./checkpoints/model.ckpt")
    print(np.argmax(sess.run(y_conv,feed_dict={x:z, keep_prob: 1.0})))
    

INFO:tensorflow:Restoring parameters from ./checkpoints/model.ckpt
0
